# Création d'une segmentation client pour Olist
Problématique : identifier différents comportements des clients afin de pouvoir faire du ciblage marketing et augmenter ainsi la rentabilité de l'entreprise

Nous disposons de plusieurs informations anonymisées de la société Olist sur :

- les commandes
- les clients
- les produits
- les catégories des produits
- des informations sur le paiment des commandes
- les vendeurs.
Il est important de noter qu'une commande peut contenir des produits de différents vendeurs.

In [1]:
#notre package de fonctionnalités
from Package import Scripts_Analyse01 as pk

import pandas as pd
import numpy as np
import missingno
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import scipy.stats
from scipy.stats import pearsonr
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from yellowbrick.features import ParallelCoordinates
from plotly.graph_objects import Layout
import jenkspy
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, scale
from sklearn import preprocessing
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.features import PCA as PCA_yellow
from yellowbrick.style import set_palette
# Import train_test_split function
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GridSearchCV
#Import knearest neighbors Classifier model
from sklearn.neighbors import KNeighborsClassifier
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_selection import SelectKBest, f_classif,mutual_info_classif
from skfeature.function.similarity_based import fisher_score
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

from hyperopt import fmin, tpe, hp, SparkTrials, STATUS_OK, Trials
 
import mlflow
from datetime import datetime

ModuleNotFoundError: No module named 'Package'

Regardons les différentes tables à notre disposition et s'il y a des doublons où des valeurs manquantes

Commençons par la table des commandes.

In [2]:
orders = pd.read_csv('./input/olist_orders_dataset.csv')

NameError: name 'pd' is not defined

In [3]:
orders.head(5)

NameError: name 'orders' is not defined

In [4]:
orders.shape

NameError: name 'orders' is not defined

In [5]:
orders[['order_id']].duplicated().sum()

NameError: name 'orders' is not defined

Nous avons une ligne par commande et il n'y a pas de doublons dans cette table.

Regardons si cette table contient des données manquantes.

In [6]:
tab=pk.del_Nan(orders, 0.0000000001,0, 0)

NameError: name 'pk' is not defined

In [7]:
tab

NameError: name 'tab' is not defined

In [8]:
pk.matrix_vm(orders, (16,8), (0.60, 0.64, 0.49))

NameError: name 'pk' is not defined

Nous observons des données manquantes pour les dates de livraison par exemple. Nous pouvons conserver ces données car ce sont surement des commandes non livrées.

In [9]:
orders.loc[pd.isna(orders["order_approved_at"])==True]

NameError: name 'orders' is not defined

Nous avons des données manquantes pour les commandes annulées, ce qui est normal car nous n'avons pas les informations. Est ce qu'il est intéressant de conserver ces données ??????

Il serait interessant de créer des variables comme le délai de livraison au client, délai de livraison au transporteur et le délai entre la livraison transporteur et la livraison client.

Nous créerons une fonction pour la création des variables et l'aggrégation ensuite. Passons à l'étude de la table reviews.

In [10]:
reviews = pd.read_csv('./input/olist_order_reviews_dataset.csv', nrows = num_rows)

NameError: name 'pd' is not defined

In [11]:
reviews.head(5)

NameError: name 'reviews' is not defined

In [12]:
reviews.shape

NameError: name 'reviews' is not defined

In [13]:
reviews[['review_id', 'order_id']].duplicated().sum()

NameError: name 'reviews' is not defined

In [14]:
tab=pk.del_Nan(reviews, 0.0000000001,0, 0)

NameError: name 'pk' is not defined

In [15]:
tab

NameError: name 'tab' is not defined

Il n'y a pas de doublons dans cette table. Les colonnes review_comment_title et review_comment_message contiennent des données vides, mais le client n'est pas obligé de remplir ces données. Nous créerons des colonnes binaires sur ces colonnes et nous ferons des aggrégations par commande par la suite. Etudions la table concernant les paiements.

In [17]:
payments = pd.read_csv('./input/olist_order_payments_dataset.csv', nrows = num_rows)

NameError: name 'pd' is not defined

In [18]:
payments.head(5)

NameError: name 'payments' is not defined

In [19]:
payments[['order_id', 'payment_sequential']].duplicated().sum()

NameError: name 'payments' is not defined

In [20]:
tab=pk.del_Nan(payments, 0.0000000001,0, 0)
tab

NameError: name 'pk' is not defined

Il n'y a pas de doublons et il n'y a pas de données manquantes dans cette table. Nous allons calculer les sommes par commandes et les moyennes. Nous créerons une fonction pour faire ce traitement. Analysons la table des produits, des catégories de produits et des vendeurs.

In [21]:
prod_transl = pd.read_csv('./input/product_category_name_translation.csv', nrows = num_rows)

NameError: name 'pd' is not defined

In [22]:
products = pd.read_csv('./input/olist_products_dataset.csv', nrows = num_rows)

NameError: name 'pd' is not defined

In [23]:
sellers = pd.read_csv('./input/olist_sellers_dataset.csv', nrows = num_rows)

NameError: name 'pd' is not defined

In [24]:
prod_transl.head(5)

NameError: name 'prod_transl' is not defined

In [25]:
prod_transl[['product_category_name']].duplicated().sum()

NameError: name 'prod_transl' is not defined

In [26]:
tab=pk.del_Nan(prod_transl, 0.0000000001,0, 0)
tab

NameError: name 'pk' is not defined

In [27]:
products.head(5)

NameError: name 'products' is not defined

In [28]:
products[['product_id']].duplicated().sum()

NameError: name 'products' is not defined

In [29]:
tab=pk.del_Nan(products, 0.0000000001,0, 0)
tab

NameError: name 'pk' is not defined

Nous pouvons remplacer les vides par 0 pour la table produit, car cela nous indiquera qu'il n'y a pas d'information pour ces produits.

In [31]:
sellers.head(5)

NameError: name 'sellers' is not defined

In [32]:
sellers[['seller_id']].duplicated().sum()

NameError: name 'sellers' is not defined

In [33]:
tab=pk.del_Nan(sellers, 0.0000000001,0, 0)
tab

NameError: name 'pk' is not defined

Il n'y a pas de doublons dans ces tables. Il existe des données manquantes dans la tables produits, mais très peu. Nous conservons les données telles quelles. Regardons la table contenant les produits par commande.

In [1]:
items = pd.read_csv('./input/olist_order_items_dataset.csv', nrows = num_rows)

NameError: name 'pd' is not defined

In [2]:
items.head(5)

NameError: name 'items' is not defined

In [3]:
items[['order_id','product_id', 'order_item_id']].duplicated().sum()

NameError: name 'items' is not defined

In [4]:
tab=pk.del_Nan(items, 0.0000000001,0, 0)
tab

NameError: name 'pk' is not defined

Il n'y a aucun doublons et aucunes données manquantes.

Regardons la table clients

In [5]:
customer_obj = pd.read_csv('./input/olist_customers_dataset.csv', nrows = num_rows)

NameError: name 'pd' is not defined

In [6]:
customer_obj

NameError: name 'customer_obj' is not defined